### Import Libraries

In [1]:
from wrapper_functions import *

C:\Users\Admin-Dell\Documents\github_projects\pop_ensemble\code\wrapper_functions.py:1250: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if kernel_type is 'hmc':
C:\Users\Admin-Dell\Documents\github_projects\pop_ensemble\code\wrapper_functions.py:1257: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if kernel_type is 'hmc':
C:\Users\Admin-Dell\Documents\github_projects\pop_ensemble\code\wrapper_functions.py:1273: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if step_adaptor_type is 'simple':
C:\Users\Admin-Dell\anaconda3\lib\site-packages\gpflow\experimental\utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.decorator.check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(
C:\Users\Admin-Dell\anaconda3\lib\site-packages\gpflow\experimental\utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.inheritance.inherit_check_shapes which is considered *experimenta

TensorFlow version: 2.11.0. Expected: 2.7.0
TensorFlow Probability version: 0.19.0. Expected: 0.15.0


### Read in Data

In [2]:
#training2010 = pd.read_csv('../data/merged_wp_census_data_280922.csv')
training2010 = pd.read_csv('../data/merged_wp_census_data2_081122.csv')
county_adj = pd.read_csv('../data/countyadj2.csv', index_col = 0)
models = ['acs', 'pep', 'worldpop']

In [7]:
data_NY, adj_NY = subset_data_by_state(training2010, county_adj, 'New York', 'NY')

data_GA, adj_GA = subset_data_by_state(training2010, county_adj, 'Georgia', 'GA')

In [8]:
data_GA

,GEOID,NAME,acs,census,pep,lon,lat,worldpop
357,13001,"Appling County, Georgia",18050,18236,18334,-82.310258,31.744120,18252.248366
358,13003,"Atkinson County, Georgia",8332,8375,8364,-82.900324,31.277997,8280.643018
359,13005,"Bacon County, Georgia",10969,11096,11062,-82.479974,31.543162,11092.873979
360,13007,"Baker County, Georgia",3657,3451,3431,-84.465345,31.310185,3476.480121
361,13009,"Baldwin County, Georgia",46905,45720,45704,-83.254069,33.068964,45423.391449
...,...,...,...,...,...,...,...,...
511,13313,"Whitfield County, Georgia",99910,102599,102745,-84.971605,34.796990,102848.123472
512,13315,"Wilcox County, Georgia",9041,9255,9314,-83.470904,31.969460,9207.866530
513,13317,"Wilkes County, Georgia",10580,10593,10388,-82.755558,33.783775,10579.815508
514,13319,"Wilkinson County, Georgia",9685,9563,9529,-83.194991,32.801053,9687.102703


# Default Configs

In [4]:
# MCMC configs.
mcmc_step_size=0.1 # @param
mcmc_sample_size=500 # @param
mcmc_num_steps=10_000 # @param
mcmc_burnin=2_500 # @param
mcmc_nchain=10 # @param
mcmc_seed=0 # @param

DEFAULT_MCMC_CONFIG = dict(step_size=mcmc_step_size, 
                           num_steps=mcmc_sample_size, 
                           burnin=mcmc_burnin, 
                           nchain=mcmc_nchain, 
                           seed=mcmc_seed)

## Bayesian Model Averaging

A Bayesian ensemble model where ensemble weights $w_k's$ are parameterized by Gaussian process priors:

$y \sim N(\mu(x), \sigma^2)$ 

$\mu(x) = \sum_{k=1}^K w_k(x) * m_k(x) \quad$  where $\{m_k\}_{k=1}^K$ are base model predictions.

$w(x) = softmax(f(x)) \qquad\;\;\;$ where $w=[w_1, \dots, w_K]$ and $f=[f_1, \dots, f_K]$

$f \stackrel{i.i.d.}{\sim} GaussianProcess(0, k)$


### Run MCMC

In [11]:
mcmc_config = DEFAULT_MCMC_CONFIG.copy()
mcmc_config.update(dict(burnin = 2500, num_steps = 10000, nchain = 5, kernel_type = 'nuts'))
mcmc_config

{'step_size': 0.1,
 'num_steps': 10000,
 'burnin': 2500,
 'nchain': 5,
 'seed': 0,
 'kernel_type': 'nuts'}

Seems like this takes ~2.6s per num_steps
Took 265s for num_steps = 100
Took 51s for num_steps = 50
102s for num_steps = 100
262s for num_steps = 100 test
12s for num_steps = 100 the improved test
and 16m for num_steps = 10000 and nchain = 5! Great. \\

I think this was before I re-computed the Q determinant every time. With that, it takes 5 hours for 10000 steps and nchain = 5 (or 10?) So ya, it's slow. I can't imagine what this will be like for the entire US. However, that could have been because of bad sleep settings on my comp, so we might be a-ok here

For NY (n = 62) nuts, nchain = 2, num_steps = 100, 10m 
For GA (n = 159) nuts, nchain = 2, num_steps = 100, 230s...? Maybe because of the 6% acceptance?
For US (n = 3064) nuts, nchain = 2, num_steps = 100, 482s. So not linear time! Yay
For US (n = 3064) nuts, nchain = 5, num_steps = 10000, 

In [ ]:
t0 = time.perf_counter()
CAR_samples, chain_samples, sampler_stat = run_mcmc_CAR(data = training2010,
                                                        adjacency = county_adj,
                                                        models = models,
                                                        debug_mode = True,
                                                        **mcmc_config)  
print(time.perf_counter() - t0)

fixing tau2 and rho
kernel type is nuts
10000


Saving and loading Python objects with Pickle

In [47]:
import pickle
with open('../data/CAR_samples_with_mat_determinant.pickle', 'wb') as results_file:
  pickle.dump(CAR_samples, results_file)

In [33]:
with open(r"../data/CAR_samples.pickle", "rb") as input_file:
     CAR_samples2 = pickle.load(input_file)

In [21]:
mcmc_nchain = 10

# I can just run this first to get the code working. This way I can skip editing posterior_inference
# and the prepare_mcmc and just focus on the log probability function
CAR_samples, _ = run_mcmc(init_state=init_state,
                             target_log_prob_fn=target_log_prob_fn,
                             **mcmc_config)  

bma_gp_w_samples = run_posterior_inference(model_dist=bma_prior, 
                                           model_config=bma_model_config,
                                           Y=Y_train, 
                                           map_config=map_config,
                                           mcmc_config=mcmc_config)


bma_joint_samples = make_bma_samples(X_test1, Y_test, base_preds_test, 
                                     bma_weight_samples=bma_gp_w_samples[0],
                                     bma_model_config=bma_model_config, 
                                     n_samples=bma_n_samples_eval, 
                                     seed=bne_seed,
                                     y_samples_only=False)



Running MAP:	13129634283520.0...13028767744.0...12917108736.0...12837866496.0...12796110848.0...12773912576.0...12757137408.0...12761999360.0...12907206656.0...13081935872.0...Done.
Running MCMC:	Acceptance Ratio: 0.6058999300003052


In [48]:
models

['acs', 'pep', 'worldpop']

In [49]:
CAR_ensemble_weights = tf.reduce_mean(CAR_samples[0], axis = 2)

weights_dict = {
    "acs": CAR_ensemble_weights[:,0],
    "pep": CAR_ensemble_weights[:,1],
    "worldpop": CAR_ensemble_weights[:,2]
}

color_weights = make_color_norm(
    list(weights_dict.values())[1],   
    method="percentile")

In [50]:
# get exponentiated values and sum across models
exp_phi = tf.math.exp(CAR_ensemble_weights)
exp_phi_rows = tf.reduce_sum(exp_phi, 1)

# get model weights and calculate mean estimate
u = exp_phi/exp_phi_rows[...,None]


In [51]:
norm_weights_dict = {
    "acs": u[:,0],
    "pep": u[:,1],
    "worldpop": u[:,2]
}

color_norm_weights = make_color_norm(
    list(norm_weights_dict.values())[1],   
    method="percentile")

In [56]:
import plotly
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
# df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
#                    dtype={"fips": str})
import matplotlib.colors as colors

import plotly.express as px

In [53]:
for model_name in models:
    output = pd.DataFrame(np.column_stack([training2010[["GEOID"]], weights_dict[model_name]]))
    output = output.set_axis(['GEOID', model_name], axis=1)
    output[model_name] = output[model_name].astype(float)
    fig = px.choropleth_mapbox(output, geojson=counties, locations='GEOID', color=model_name,
                           color_continuous_scale="Viridis",
                           #range_color=(0.05,0.07),
                           mapbox_style="carto-positron",
                           #featureidkey="properties.MWS_ID",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'}
                          )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

NameError: name 'counties' is not defined

In [54]:
for model_name in models:
    output = pd.DataFrame(np.column_stack([training2010[["GEOID"]], norm_weights_dict[model_name]]))
    output = output.set_axis(['GEOID', model_name], axis=1)
    output[model_name] = output[model_name].astype(float)
    fig = px.choropleth_mapbox(output, geojson=counties, locations='GEOID', color=model_name,
                           color_continuous_scale="Viridis",
                           #range_color=(0.05,0.07),
                           mapbox_style="carto-positron",
                           #featureidkey="properties.MWS_ID",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'}
                          )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

NameError: name 'counties' is not defined

! conda install geopandas==0.3.0
! conda install pyshp==1.2.10
! conda install shapely==1.6.3